<h1 align="center">Sistema de Recuperação de Informações</h1>

<h2 align="center"> Lista de Exercícios 1 </h2>

<h3 align="center"> Gabriel Lima Novais</h3>

___

Todos os exercícios requerem uma implementação funcional do problema, no corpo do notebook. Para cada exercício consultas de teste devem ser propostas para demonstrar que a implementação atende aos requisitos do exercício.
As resposta devem ser enviadas como um notebook (.ipynb) para o professor.

### 1. Importando pacotes

In [4]:
import numpy as np
import pandas as pd
import os
import re
import whoosh
import nltk
import string
import matplotlib.pyplot as plt
import enchant

from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh import qparser
from whoosh.qparser import QueryParser
from collections import defaultdict


%matplotlib inline

### 2. Configurando Corpus

In [5]:
#Colocando o diretório:
corpus = "/home/novais/Desktop/Mestrado/3_trim_2019/SRI/corpusEx.csv"

#Importando os dados da Wikipedia em formato csv:
df = pd.read_table(corpus, sep=",", nrows=50000,header=None)

#Tratando a base de dados:
df.columns = ['id','content']
df.id = df.id.apply(lambda x: int(x[10:]))

#Retirando uma amostra pequena para realizar as questões da lista sem demorar muito...
#OBS: Toda vez que rodar essa linha abixo o Corpus vai mudar e assim as respostas devem mudar!!!
corpus =  df.sample(n=100)
corpus = corpus.reset_index().drop(['index'], axis=1)

#Visualizando aas primeiras linhas do Corpus:
corpus.head()

,id,content
0,3016425,Asia Series The Asia Series is an internatio...
1,3210816,HAT medium HAT Medium (hypoxanthine-aminopte...
2,3800929,"Flavor of the Weak ""Flavor of the Weak"" is a..."
3,5184327,Thrill Seeker Thrill Seeker is the debut ful...
4,1115482,Streetcar suburb A streetcar suburb is a res...


In [6]:
#Configurações iniciais:
nltk.download('stopwords')
swu = stopwords.words('english')+ list(string.punctuation)
stopwords.words('english')
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /home/novais/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Preparando o corpus com e sem *stemming*

In [8]:
x = corpus.content

Com_Stemming = []
Sem_Stemming = []

for texto in x:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    Com_Stemming.append(tlimpo)
    tlimpo1 = [token.lower() for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    Sem_Stemming.append(tlimpo1)

In [11]:
print(Com_Stemming[0][50:60])
print(Sem_Stemming[0][50:60])

['kbo', 'korean', 'seri', 'chines', 'profession', 'basebal', 'leagu', 'cpbl', 'taiwan', 'seri']
['kbo', 'korean', 'series', 'chinese', 'professional', 'baseball', 'league', 'cpbl', 'taiwan', 'series']


### Exercício 1: Truncagem e revocação.

Baseando-se no indice invertido construído na prática 1, calcule a diferença de revocação com e sem a utilização de "stemming", ou truncagem na construção do índice.

In [51]:
#Função search:
def search(word,where):
    
    #word é a palavra que deseja buscar.
    #where é em que coluna, na de stemming ou sem stemming.
    
    indice = defaultdict(lambda:set([]))
    for tid,t in enumerate(where):
        for term in t:
            indice[term].add(tid)
    return indice[(word.lower())]  

In [52]:
#Exemplo de Funcionamento:
print("Resultados com stemming: " + str(search('heat',Com_Stemming)))
print("Resultados sem stemming: " + str(search('heat',Sem_Stemming)))

Resultados com stemming: {0, 35, 38, 12, 46, 18, 90, 93}
Resultados sem stemming: {0, 12, 46, 90, 93}


In [53]:
def PR(recuperados, relevantes):
    
    #cálculo da Previsão e da Revocação
    
    previsao = len(recuperados &  relevantes)/len(recuperados)
    revocacao = len(recuperados &  relevantes)/len(relevantes)
    
    return previsao,revocacao

In [89]:
# Palavra a ser consultada
word = 'walk'

In [123]:
# Resultados da busca
sem_st = search(word,Sem_Stemming)
com_st = search(stemmer.stem(word), Com_Stemming)

print('Quantidade de Resultados com Stemming: ' + str(len(com_st)))

print('Quantidade de Resultados sem Stemming: ' + str(len(sem_st)))


Quantidade de Resultados com Stemming: 7
Quantidade de Resultados sem Stemming: 3


In [91]:
# A revocação é calculada como a proporção entre o resultado de stemming com o resultado sem stemming.
print('Precisão: ' +  str(PR(sem_st, com_st)[0]))
print('Revocação: ' + str(PR(sem_st, com_st)[1]))

Precisão: 1.0
Revocação: 0.42857142857142855


### Exercício 2: Expansão de consultas
Crie grupos de equivalência para alguns termos de busca e calcule a diferença em termos de revocação e, possivelmente precisão, na resposta a consultas expandidas e não expandidas. Dica: use tempos verbais, pluralização, sinônimos, etc.

In [92]:
words = ['walk', 'walker', 'walkers', 'walking', 'walked']

In [93]:
res_sem_st = set()
res_com_st = set()

print(' Word - Sem Stemming - Com Stemming')
print('-----------------------------------')
for word in words:
    res1 = search(word,Sem_Stemming)
    res2 = search(word,Com_Stemming)
    res_sem_st.update(res1)
    res_com_st.update(res2)
    print(' {}: {} - {}'.format(word, len(res1),len(res2)))
print('\n')
print('TOTAL: {} - {}'.format(len(res_sem_st),len(res_com_st)))

 Word - Sem Stemming - Com Stemming
-----------------------------------
 walk: 3 - 7
 walker: 0 - 0
 walkers: 0 - 0
 walking: 4 - 0
 walked: 1 - 0


TOTAL: 6 - 7


In [94]:
print('Precisão: ' +  str(PR(res_sem_st, res_com_st)[0]))
print('Revocação: ' + str(PR(res_sem_st, res_com_st)[1]))

Precisão: 1.0
Revocação: 0.8571428571428571


###  Exercício 3: Verificação ortográfica
Implemente uma expansão de consulta por meio da correção ortográfica. Utilize o corretor ortográfico [Pyenchant](http://pythonhosted.org/pyenchant/) para fazer as correções.

In [153]:
#Contnuando com o exemplo que usamos acima
word='wlk'

In [154]:
#Função que procura cada uma das palavras que são possíveis de estarem certas, caso não esteja certa.

def searchReview(word,ind):
    
    #selecionando o dicionário na língua inglesa:
    d = enchant.Dict("en_US")
    
    #caso a palavra esteja errada:
    if not d.check(word):
        print("É possível que você tenha escrito errado...")
        print("Palavras possíveis: ")
        ans = set()
        
        #percorrendo cada palavra na sugestão:
        for i in d.suggest(word):
            print("- "+i)
            aux = search(i,ind)
            #adiciona palavra possível no conjunto resposta
            ans.update(aux)
            
    #caso a palavra esteja correta:
    else:
        ans = search(word,ind)
        
    return ans

In [155]:
#verificando a função acima:
searchReview(word,Sem_Stemming)

É possível que você tenha escrito errado...
Palavras possíveis: 
- elk
- wk
- walk
- ilk
- wok


{4, 32, 78}

In [156]:
#verificando a função acima:
searchReview(word,Com_Stemming)

É possível que você tenha escrito errado...
Palavras possíveis: 
- elk
- wk
- walk
- ilk
- wok


{4, 12, 32, 78, 86, 96, 99}

In [159]:
# Resultados da busca simples sem correção ortográfica
sem_st = search(word,Sem_Stemming)
com_st = search(stemmer.stem(word), Com_Stemming)

print('Quantidade de Resultados com Stemming: ' + str(len(com_st)))

print('Quantidade de Resultados sem Stemming: ' + str(len(sem_st)))

Quantidade de Resultados com Stemming: 0
Quantidade de Resultados sem Stemming: 0


In [162]:
# Resultados da busca com correção ortográfica
sem_st = searchReview(word,Sem_Stemming)
com_st = searchReview(stemmer.stem(word), Com_Stemming)

print('Quantidade de Resultados com Stemming: ' + str(len(com_st)))

print('Quantidade de Resultados sem Stemming: ' + str(len(sem_st)))

É possível que você tenha escrito errado...
Palavras possíveis: 
- elk
- wk
- walk
- ilk
- wok
É possível que você tenha escrito errado...
Palavras possíveis: 
- elk
- wk
- walk
- ilk
- wok
Quantidade de Resultados com Stemming: 7
Quantidade de Resultados sem Stemming: 3


###  Exercício 4: Consultas por frases
Implemente um indice invertido que permita consulta por frases, conforme definido na aula 2.

In [164]:
#Criando novo indice para consulta por frases
x = corpus.content

Com_Stemming = []
Sem_Stemming = []

for texto in x:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    Com_Stemming.append(tlimpo)
    tlimpo1 = [token.lower() for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    Sem_Stemming.append(tlimpo1)

indice_frase_CS = defaultdict(lambda: defaultdict(list))
indice_frase_SS = defaultdict(lambda: defaultdict(list))

for texto_id, texto in enumerate(Com_Stemming):
    for token_id, token in enumerate(texto):
        indice_frase_CS[token][texto_id].append(token_id)
for texto_id, texto in enumerate(Sem_Stemming):
    for token_id, token in enumerate(texto):
        indice_frase_SS[token][texto_id].append(token_id)

In [170]:
#Testando o índice:
indice_frase_CS[stemmer.stem("walk")]

defaultdict(list,
            {4: [282, 380, 720, 861, 1215],
             12: [703],
             32: [51, 422, 917],
             78: [1049],
             86: [437],
             96: [520],
             99: [427]})

In [174]:
len(indice_frase_CS[stemmer.stem("Walk")][4])

5

In [233]:
#Função de busca de frases:
def searchPhrase(sentence, ind):
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(sentence) if token not in swu]
 
    matches = defaultdict(lambda:defaultdict(lambda:set([])))
    for token in tokens:
        matches[token] = ind[token]
    
    doc_pos = set(matches[tokens[0]])
    for tok in tokens:
        doc_pos = doc_pos.intersection(set(matches[token]))
        
    ans = set([]) 
    for doc in doc_pos:
        for pos in matches[tokens[0]][doc]:
            if all((pos + token_id + 1 in matches[token][doc]) for token_id, token in enumerate(tokens[1:len(tokens)])):
                ans.add(doc) 
                
    return ans

In [234]:
#Exemplo de utilização:
searchPhrase("take them to walk two miles", indice_frase_CS)
# Ou seja, o que encontramos foi justamente a linha no qual o corpus possuia 
# em seu content exatamente a frase acima.

{4}

In [235]:
"take them to walk two miles" in corpus.content[4]

True

###  Exercício 5: Consulta híbrida.
Modifique a solução acima para permitir respostas alternativas caso a frase não retorne resultados. Por exemplo, retornar, documentos que contenham parte da frase, ou uma busca booleana simples combinando as palavras da frase.

In [451]:
 def whooshSearcher(sentence, ind, book):
        
    schema = Schema(cont=TEXT(phrase=True, stored=True))
    x = book
    
    if os.path.exists('indexdir'):
        ix = open_dir('indexdir')
    else:
        os.mkdir('indexdir')
        ix = create_in("indexdir", schema)
        writer = ix.writer()
        for txt in x:
            writer.add_document(cont=txt)
        writer.commit()
        
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(sentence) if token not in swu]
 
    ans = searchPhrase(sentence, ind)
                
    if ans == set([]):
        query = QueryParser("cont", schema=ix.schema,group=qparser.OrGroup)
        q = query.parse(sentence)
        s = ix.searcher()
        ans_near = s.search(q)
        return ans_near
    else:
        return ans

In [452]:
# Sabemos que a frase "take them to walk two miles" existe no corpus.content[4], vamos ver se
# a frase similar "walk four miles" consegue ser encontrada e se nos resultados o corpus.content[4] aparece...
result = whooshSearcher("walk four miles", indice_frase_CS,corpus.content)

In [454]:
#Os quatros primeiros resultados da pesquisa acima
result[0:5]

[<Hit {'cont': ' Streetcar suburb  A streetcar suburb is a residential community whose growth and development was strongly shaped by the use of streetcar lines as a primary means of transportation. Early suburbs were served by horsecars, but by the late 19th century cable cars and electric streetcars, or trams, were used, allowing residences to be built further away from the urban core of a city. Streetcar suburbs, usually called additions or extensions at the time, were the forerunner of today\'s suburbs in the United States and Canada. Although most closely associated with the electric streetcar, the term can be used for any suburb originally built with streetcar-based transit in mind, thus some streetcar suburbs date from the early 19th century. As such, the term is general and one development called a streetcar suburb may vary greatly from others. However, some concepts are generally present in streetcar suburbs, such as straight (often gridiron) street plans and relatively narrow 

In [246]:
# Pelo jeito tradicional:
"walk miles" in corpus.content[4]

False

In [455]:
#Logo concluímos que a busca pelo Whoosh é melhor...

#### FIM